In [3]:
import numpy as np

# Sample liquidity pool data (example values)
initial_price = 0.0146  # Initial price of ALI in USD
initial_liquidity_ali = 100000000  # Initial liquidity of ALI tokens in the pool
initial_liquidity_usd = initial_liquidity_ali * initial_price  # Equivalent USD liquidity

# Parameters for the scenario
buy_amount_usd = 200_000  # USD amount to buy ALI
fee_rate = 0.003  # Trading fee rate (0.3%)

# Calculate the amount of ALI bought and price impact
def calculate_price_impact(buy_amount_usd, initial_liquidity_ali, initial_price):
    buy_amount_ali = buy_amount_usd / initial_price
    new_liquidity_ali = initial_liquidity_ali - buy_amount_ali
    new_price = (initial_liquidity_usd + buy_amount_usd) / new_liquidity_ali
    price_impact = new_price - initial_price
    return buy_amount_ali, new_price, price_impact

# Perform the trade and calculate impermanent loss
buy_amount_ali, new_price, price_impact = calculate_price_impact(buy_amount_usd, initial_liquidity_ali, initial_price)
impermanent_loss = 2 * (np.sqrt((initial_liquidity_ali - buy_amount_ali) / initial_liquidity_ali) - 1)

# Calculate fees and net result
trade_fee = buy_amount_usd * fee_rate
total_cost_usd = buy_amount_usd + trade_fee
final_value_usd = (buy_amount_ali * new_price) - trade_fee
net_gain_loss = final_value_usd - total_cost_usd

print(f"Buy Amount ALI: {buy_amount_ali}")
print(f"New Price of ALI: {new_price}")
print(f"Price Impact: {price_impact}")
print(f"Impermanent Loss: {impermanent_loss}")
print(f"Trade Fee: {trade_fee}")
print(f"Total Cost USD: {total_cost_usd}")
print(f"Final Value USD: {final_value_usd}")
print(f"Net Gain/Loss USD: {net_gain_loss}")


Buy Amount ALI: 13698630.1369863
New Price of ALI: 0.019234920634920636
Price Impact: 0.004634920634920636
Impermanent Loss: -0.1420293881440029
Trade Fee: 600.0
Total Cost USD: 200600.0
Final Value USD: 262892.0634920635
Net Gain/Loss USD: 62292.06349206349


In [4]:
# Initial liquidity data
initial_ali = 17_400_000  # ALI tokens
initial_eth = 39.03  # ETH

# Current price of ETH in USD (you can fetch the current price from a reliable source)
eth_price_usd = 3103  # Example: $2000 per ETH

# Calculate the initial price of ALI in ETH
initial_price_ali_eth = initial_eth / initial_ali
print(f"Initial Price of ALI in ETH: {initial_price_ali_eth:.8f}")

# USD amount to be used for buying ALI
buy_amount_usd = 200_000

# Convert USD to ETH
buy_amount_eth = buy_amount_usd / eth_price_usd
print(f"Buy Amount in ETH: {buy_amount_eth:.8f}")

# Simulate the purchase (constant product formula: x * y = k)
new_eth = initial_eth + buy_amount_eth
new_ali = initial_ali * initial_eth / new_eth

# Calculate the amount of ALI bought
amount_ali_bought = initial_ali - new_ali
print(f"Amount of ALI Bought: {amount_ali_bought:.8f}")

# New price of ALI in ETH after purchase
new_price_ali_eth = new_eth / new_ali
print(f"New Price of ALI in ETH: {new_price_ali_eth:.8f}")

# Calculate price impact
price_impact = (new_price_ali_eth - initial_price_ali_eth) / initial_price_ali_eth * 100
print(f"Price Impact: {price_impact:.2f}%")

# Calculate impermanent loss
impermanent_loss = 2 * (np.sqrt((initial_ali * initial_eth) / (new_ali * new_eth)) - 1)
print(f"Impermanent Loss: {impermanent_loss:.8f}")

# Include fees and gas costs if needed for a more comprehensive analysis


Initial Price of ALI in ETH: 0.00000224
Buy Amount in ETH: 64.45375443
Amount of ALI Bought: 10837404.70441150
New Price of ALI in ETH: 0.00001577
Price Impact: 602.99%
Impermanent Loss: 0.00000000


In [5]:
import requests

# Define the GraphQL endpoint and query
url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
query = """
{
  pool(id: "POOL_ADDRESS") {
    id
    token0 {
      id
      symbol
    }
    token1 {
      id
      symbol
    }
    liquidity
    token0Price
    token1Price
    volumeToken0
    volumeToken1
  }
}
"""

# Replace with the actual pool address
pool_address = "0xF260d15e8eBe54D210ef53F5b61Cb46bD9Aa29EE"  # Replace with the ALI/USDC or ALI/WETH pool address
query = query.replace("POOL_ADDRESS", pool_address)

# Send the request to The Graph
response = requests.post(url, json={'query': query})
data = response.json()

# Extract and print the liquidity details
pool_data = data['data']['pool']
print("Pool ID:", pool_data['id'])
print("Token0:", pool_data['token0']['symbol'])
print("Token1:", pool_data['token1']['symbol'])
print("Liquidity:", pool_data['liquidity'])
print("Token0 Price:", pool_data['token0Price'])
print("Token1 Price:", pool_data['token1Price'])
print("Volume Token0:", pool_data['volumeToken0'])
print("Volume Token1:", pool_data['volumeToken1'])


TypeError: 'NoneType' object is not subscriptable

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
def get_uniswap_top_pools(n_pools):
    
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

    # GraphQL query to fetch the pool id and daily volume for the top n_pools
    query = """
    {{
      pools(first: {0}, orderBy: volumeUSD, orderDirection: desc) {{
        id
        volumeUSD
      }}
    }}
    """.format(n_pools)

    # Make the request
    response = requests.post(url, json={'query': query})

    # Get the JSON data from the response
    data = json.loads(response.text)

    # Get the pool IDs from the data
    pool_ids = [pool['id'] for pool in data['data']['pools']]

    return pool_ids

def get_uniswap_pool_day_data(pool_id, number_of_days):
  
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
    query = f"""
    {{
        poolDayDatas(where: {{ pool: "{pool_id}" }}, first: {number_of_days}, orderBy: date, orderDirection: desc) {{
            date
            tick
            sqrtPrice
            liquidity
            volumeUSD
            volumeToken0
            volumeToken1
            tvlUSD
            feesUSD
            close
            open
            low
            high
        }}
    }}
    """
    response = requests.post(url, json={'query': query})
    data = response.json()
    df = pd.DataFrame(data['data']['poolDayDatas'])

    return df

def save_data_to_csv(data, file_path):
    keys = data[0].keys()
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)


def preprocess_uniswap_subgraph_data(data_df):
    data_df['volumeUSD'] = data_df['volumeUSD'].astype(float)
    data_df['volumeToken0'] = data_df['volumeToken0'].astype(float)
    data_df['volumeToken1'] = data_df['volumeToken1'].astype(float)
    data_df['sqrtPrice'] = (data_df['sqrtPrice'].astype(float))
    data_df['liquidity'] = data_df['liquidity'].astype(float)
    data_df['tvlUSD'] = data_df['tvlUSD'].astype(float)
    data_df['feesUSD'] = data_df['feesUSD'].astype(float)
    data_df['close'] = data_df['close'].astype(float)
    data_df['open'] = data_df['open'].astype(float)
    data_df['low'] = data_df['low'].astype(float)
    data_df['high'] = data_df['high'].astype(float)
    
      # Convert date from UNIX timestamp to datetime format
    data_df['date'] = pd.to_datetime(data_df['date'], unit='s')
    # 7 days Rolling averages
    data_df['feesUSD'] = data_df['feesUSD'].rolling(window=7, min_periods=1).mean()

    data_df['date'] = pd.to_datetime(data_df['date'], unit='s')
  
# Check for missing data in the control group
    missing_values = data_df.isnull().sum()
    missing_control = missing_values[missing_values > 0]

    return data_df

def explore_uniswap_pool_data(df):
    #Raw data analysis

    df.plot(x='date', y='sqrtPrice')

    df.plot(x='date', y='feesUSD')
    df.plot(x='date', y='liquidity')

    ax = df.plot(x='date', y='liquidity', color='blue', label='liquidity')
    df.plot(x='date', y='sqrtPrice', color='red', secondary_y=True, ax=ax, label='sqrtPrice')

    ax.set_ylabel('liquidity')
    ax.right_ax.set_ylabel('sqrtPrice')
    plt.title('liquidity and sqrtPrice Over Time')
    plt.show()

    ax = df.plot(x='date', y='liquidity', color='blue', label='liquidity')
    df.plot(x='date', y='feesUSD', color='red', secondary_y=True, ax=ax, label='Fees')

    ax.set_ylabel('liquidity')
    ax.right_ax.set_ylabel('Fees')
    plt.title('liquidity and Fees Over Time')
    plt.show()

    ax = df.plot(x='date', y='volumeToken0', color='blue', label='volumeToken0')
    df.plot(x='date', y='volumeToken1', color='red', secondary_y=True, ax=ax, label='volumeToken1')

    ax.set_ylabel('volumeToken0')
    ax.right_ax.set_ylabel('volumeToken1')
    plt.title('liquidity and Fees Over Time')
    plt.show()

    ax = df.plot(x='date', y='sqrtPrice', color='blue', label='sqrtPrice')
    df.plot(x='date', y='sqrtPrice', color='red', secondary_y=True, ax=ax, label='sqrtPrice')
    ax.set_ylabel('sqrtPrice')
    ax.right_ax.set_ylabel('sqrtPrice')
    plt.title('liquidity and Fees Over Time')
    plt.show()

    #EDA
    plt.hist(df['volumeUSD'])
    plt.show()
    sns.boxplot(x=df['volumeUSD'])
    plt.show()
    plt.scatter(x=df['date'],y=df['volumeUSD'])
    plt.show()
    sns.countplot(df['volumeUSD'])
    plt.show()
    sns.heatmap(df.corr())
    plt.show()
    corr_matrix=df.corr()
    sns.heatmap(corr_matrix,annot=True,cmap='coolwarm')

    #zscore
    df['zscore']=zscore(df['volumeUSD'])
    outliers=df[(df['zscore']>3) | (df['zscore']<-3)]
    print(outliers)

    #transformations
    df['log_transformed']=np.log(df['volumeUSD'])

    #standardization
    scaler = StandardScaler()
    df['standardized_volume'] = scaler.fit_transform(df['volumeUSD'].values.reshape(-1, 1))

    return True

In [11]:
df=get_uniswap_pool_day_data("0xF260d15e8eBe54D210ef53F5b61Cb46bD9Aa29EE",10)

In [12]:
df

""


In [7]:
# Given data
initial_price_usdc_per_ali = 0.01495
usdc_spent = 200_000
ali_received = 7925280
price_impact_percent = 42  # Positive impact means price increase

# Calculate the initial amount of ALI in the pool
initial_ali_in_pool = ali_received / (price_impact_percent / 100)
initial_usdc_in_pool = initial_ali_in_pool * initial_price_usdc_per_ali

# Calculate the new amount of ALI and USDC in the pool after the purchase
new_ali_in_pool = initial_ali_in_pool - ali_received
new_usdc_in_pool = initial_usdc_in_pool + usdc_spent

# Calculate the new price of ALI in USDC after the purchase
new_price_usdc_per_ali = new_usdc_in_pool / new_ali_in_pool

# Display the results
print(f"Initial ALI in Pool: {initial_ali_in_pool:.2f}")
print(f"Initial USDC in Pool: {initial_usdc_in_pool:.2f}")
print(f"New ALI in Pool: {new_ali_in_pool:.2f}")
print(f"New USDC in Pool: {new_usdc_in_pool:.2f}")
print(f"New Price of ALI in USDC: {new_price_usdc_per_ali:.8f}")


Initial ALI in Pool: 18869714.29
Initial USDC in Pool: 282102.23
New ALI in Pool: 10944434.29
New USDC in Pool: 482102.23
New Price of ALI in USDC: 0.04404999


In [8]:
# Initial prices and liquidity
initial_price_usdc_per_ali_pool1 = 0.01495
initial_price_usdc_per_ali_pool2 = 0.01495

# New price in Pool 1 after purchase
new_price_usdc_per_ali_pool1 = 0.04

# Assume initial liquidity in both pools for simplicity
initial_ali_in_pool1 = 17_400_000
initial_ali_in_pool2 = 17_400_000

# Simulate arbitrage until prices converge
# For simplicity, we assume equal initial liquidity and no fees

def arbitrage_simulation(price_pool1, price_pool2, liquidity_pool1, liquidity_pool2):
    while abs(price_pool1 - price_pool2) > 0.0001:  # Stop when prices converge closely
        if price_pool1 > price_pool2:
            # Arbitrage buy from Pool 2 (cheaper) and sell to Pool 1 (expensive)
            trade_amount = min(liquidity_pool2 / 100, liquidity_pool1 / 100)  # Trade 1% of pool liquidity for simplicity
            liquidity_pool2 -= trade_amount
            liquidity_pool1 += trade_amount
        else:
            # Arbitrage buy from Pool 1 (cheaper) and sell to Pool 2 (expensive)
            trade_amount = min(liquidity_pool1 / 100, liquidity_pool2 / 100)  # Trade 1% of pool liquidity for simplicity
            liquidity_pool1 -= trade_amount
            liquidity_pool2 += trade_amount
        
        # Update prices
        price_pool1 = 1 / (liquidity_pool1 / 17_400_000)
        price_pool2 = 1 / (liquidity_pool2 / 17_400_000)
    
    return price_pool1, price_pool2

final_price_pool1, final_price_pool2 = arbitrage_simulation(new_price_usdc_per_ali_pool1, initial_price_usdc_per_ali_pool2, initial_ali_in_pool1, initial_ali_in_pool2)

print(f"Final Price in Pool 1: {final_price_pool1:.8f} USDC/ALI")
print(f"Final Price in Pool 2: {final_price_pool2:.8f} USDC/ALI")


Final Price in Pool 1: 0.99999799 USDC/ALI
Final Price in Pool 2: 1.00000201 USDC/ALI


In [ ]:
import requests
import numpy as np

# Define the GraphQL endpoint and query
url = ""
query = """
{
  pool(id: "POOL_ADDRESS") {
    id
    token0 {
      id
      symbol
    }
    token1 {
      id
      symbol
    }
    liquidity
    sqrtPrice
    tick
    ticks {
      tickIdx
      liquidityNet
      liquidityGross
    }
  }
}
"""

# Replace with the actual pool address
pool_address = "0x...POOL_ADDRESS..."  # Replace with the ALI/USDC pool address
query = query.replace("POOL_ADDRESS", pool_address)

# Send the request to The Graph
response = requests.post(url, json={'query': query})
data = response.json()

# Extract pool data
pool_data = data['data']['pool']
liquidity = int(pool_data['liquidity'])
sqrt_price = float(pool_data['sqrtPrice'])
current_tick = int(pool_data['tick'])
ticks = pool_data['ticks']

# Helper function to convert sqrtPrice to price
def sqrt_price_to_price(sqrt_price):
    return (sqrt_price / (2 ** 96)) ** 2

# Calculate initial price
initial_price = sqrt_price_to_price(sqrt_price)
print(f"Initial Price: {initial_price:.8f} USDC/ALI")

# Convert buy amount from USDC to ALI
buy_amount_usdc = 200_000
current_price = initial_price
buy_amount_ali = buy_amount_usdc / current_price

# Simulate the trade
def simulate_trade(current_price, buy_amount_ali, ticks, liquidity):
    remaining_buy_amount = buy_amount_ali
    new_price = current_price
    for tick in ticks:
        if remaining_buy_amount <= 0:
            break
        tick_liquidity = int(tick['liquidityGross'])
        price_range = new_price - current_price
        if tick_liquidity > 0:
            price_increase = remaining_buy_amount / tick_liquidity
            new_price += price_increase
            remaining_buy_amount -= price_increase * tick_liquidity

    return new_price

new_price = simulate_trade(current_price, buy_amount_ali, ticks, liquidity)
price_impact = (new_price - current_price) / current_price * 100
print(f"New Price: {new_price:.8f} USDC/ALI")
print(f"Price Impact: {price_impact:.2f}%")
